**Resources**
<!-- {{< youtube akZ8JJ4gGLs >}} -->
<!-- {{< tweet 1266579585417613312 >}} -->
- [Smoothstep](https://smoothstep.io/)
- Breadth First Search
- Dijkstra's algorithm/Fast Marching Method for solving the Eikonal equation?
- BFS is generalized as Disjkstra, which is generalized as Fast Marching, then as Ordered Upwind method, then as Anisotropic Fast Marching
- Hex mazes
- Bellman-Ford Algorithm
- Fast Marching Algorithm (FMM), eikonal equation
- [Maze Art](https://troika.uk.com/work/troika-labyrinth/)
- [Lee Algorithm](https://en.wikipedia.org/wiki/Lee_algorithm)
- [Procedural Content Generation: Mazes](http://pcg.wikidot.com/pcg-algorithm:maze)
- [Wikipedia Maze Generation Algorithms](https://en.wikipedia.org/wiki/Maze_generation_algorithm)
- [Smart Move: Intelligent Path Finding](https://www.gamedeveloper.com/programming/smart-move-intelligent-path-finding)
- [Toward more Realistic Path Finding](https://www.gamedeveloper.com/programming/toward-more-realistic-pathfinding)
- [AI Wisdom A* Articles](http://www.aiwisdom.com/ai_astar.html)


Solving a Maze
- [Maze Solving Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm)


Per wikipedia:
> The best-known rule for traversing mazes is the wall follower, also known as either the left-hand rule or the right-hand rule. If the maze is simply connected, that is, all its walls are connected together or to the maze's outer boundary, then by keeping one hand in contact with one wall of the maze the solver is guaranteed not to get lost and will reach a different exit if there is one; otherwise, the algorithm will return to the entrance having traversed every corridor next to that connected section of walls at least once. The algorithm is a depth-first in-order tree traversal.

For autonomous agents I perhaps one of these will be suitable.
- [Maze Routing Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Maze-routing_algorithm)
- [Shortest Path Algorithms](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Shortest_path_algorithm)

Game Programming Gems 1 (PDF)
- Simple Implementation: Page 248
- Optimized Implementation: Page 279 
- Fuzzy Logic for Video Games: Page 313
- A Neural Net Primer: Page 324

In [263]:
# Load python code.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [264]:
# All imports
from __future__ import annotations

from heapq import heappop, heappush
from IPython.display import display
from ipycanvas import Canvas,  hold_canvas, MultiCanvas
import random
import time
from typing import Dict, List, Set, Tuple, Union
import itertools

from generation.structures import Corner, Point
from generation.maze import Maze, MazeCell
from generation.generators.random_backtracer import generate_maze_walls
from generation.npc import Agent
from generation.renderers.units import AGENT_SIZE, ROOM_SIZE_WIDTH, ROOM_SIZE_HEIGHT
from generation.renderers.wall_drawer import draw_maze, animate_drawing_by_rooms
from generation.renderers.agents_renderer import draw_agents
from generation.direction import Direction, DIR_ORIENTATION, Orientation
from generation.walkers.clueless import clueless_walk
from generation.walkers.wall_follower import wall_follower_walk
from generation.walkers.a_star import find_path as find_path_with_a_star, Path

In [265]:
def draw_path(path: List[Point], canvas: Canvas) -> None:
  canvas_points = []

  # Create an array of tuples for canvas to render in a single draw call.
  print(f'Path has {len(path)} steps')
  for location in path:
    # Find the upper left corner for the room.
    upper_left_corner = Corner(location.x * ROOM_SIZE_WIDTH, location.y * ROOM_SIZE_HEIGHT)

    # Find the midpoint of the room.
    horizontal_offset = ROOM_SIZE_WIDTH/2.0
    vertical_offset = ROOM_SIZE_HEIGHT/2.0
    midpoint = Point(upper_left_corner.x + horizontal_offset, upper_left_corner.y + vertical_offset)

    canvas_points.append((midpoint.x, midpoint.y))
  
  canvas.stroke_style = 'red'
  canvas.stroke_lines(canvas_points)

In [266]:
def draw_hud(canvas: Canvas, frame) -> None:
  LINE_HEIGHT = 14
  FIRST_LINE = 20
  HORIZONTAL_OFFSET = 450
  with hold_canvas(canvas):
    canvas.text_baseline = "top"
    canvas.clear()

    # Draw the frame count
    canvas.fill_style = 'black'
    canvas.fill_text(f'Frame: {frame}', HORIZONTAL_OFFSET, FIRST_LINE)

    # Draw Wall Walker Legend
    canvas.fill_style = 'blue'
    canvas.fill_rect(HORIZONTAL_OFFSET,FIRST_LINE+LINE_HEIGHT, AGENT_SIZE)
    canvas.fill_style = 'black'
    canvas.fill_text(f'- Wall Follower', HORIZONTAL_OFFSET + AGENT_SIZE + 5, FIRST_LINE+LINE_HEIGHT)

    # Draw Clueless Walker Legend
    canvas.fill_style = 'green'
    canvas.fill_rect(HORIZONTAL_OFFSET,FIRST_LINE+LINE_HEIGHT*2, AGENT_SIZE)
    canvas.fill_style = 'black'
    canvas.fill_text(f'- Clueless Walker', HORIZONTAL_OFFSET + AGENT_SIZE + 5, FIRST_LINE+LINE_HEIGHT*2)

    # Draw A* Walker Legend
    canvas.fill_style = 'yellow'
    canvas.fill_rect(HORIZONTAL_OFFSET,FIRST_LINE+LINE_HEIGHT*3, AGENT_SIZE)
    canvas.fill_style = 'black'
    canvas.fill_text(f'- A* Walker', HORIZONTAL_OFFSET + AGENT_SIZE + 5, FIRST_LINE+LINE_HEIGHT*3)

In [268]:
from collections.abc import Callable

def build_path_walker(path_to_walk: Path):
  path = path_to_walk
  current_step_index = -1

  def walk_path(agent: Agent, maze: Maze) -> None:
    nonlocal current_step_index, path

    if current_step_index < (len(path) - 1):
      current_step_index += 1
      agent.move_to(path[current_step_index])

  return walk_path

# 1000/125 = 8 FPS
SLEEP_TIME_SEC:float = 0.125

# Generate a maze.
maze: Maze = Maze(20, 20)
generate_maze_walls(maze)

# Create 4 layers of canvases. 0: Maze, 1: A* Path, 2: Agents, 3: HUD
mc = MultiCanvas(n_canvases=4, width=800, height=400)
display(mc)

# Create NPCs with different strategies
common_starting_point = Point(int(maze.width/2), int(maze.height/2))
wall_follower = Agent('blue')
wall_follower.maze_strategy(wall_follower_walk)
wall_follower.move_to(common_starting_point)
wall_follower.face(Direction.SOUTH)

random_walker = Agent('green')
random_walker.maze_strategy(clueless_walk)
random_walker.move_to(common_starting_point)
random_walker.face(Direction.SOUTH)

# Calculate a path using A*
a_star_walker = Agent('yellow')
a_star_walker.move_to(common_starting_point)
a_star_walker.face(Direction.SOUTH)

found_path, escape_path = find_path_with_a_star(a_star_walker, maze, maze.exit_cell.location)
if not found_path:
  raise Exception('Failed to find a path.') 

path_walker = build_path_walker(escape_path)
a_star_walker.maze_strategy(path_walker)

agents = [wall_follower, random_walker, a_star_walker]

# Initial Render
draw_maze(maze, mc[0])
draw_path(escape_path, mc[1])
draw_agents(agents, mc[2])

for frame in range(200):
  for agent in agents:
    agent.explore(maze)
  draw_agents(agents, mc[2])
  draw_hud(mc[3],frame)
  time.sleep(SLEEP_TIME_SEC)

Exit cell: Point(x=12, y=19)


MultiCanvas(height=400, width=800)

Path has 76 steps


KeyboardInterrupt: 

In [ ]:
# The Main Entry Point
"""
SAVE_ANIMATED_GIF = False

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
thread = RenderingThread(renderer, SAVE_ANIMATED_GIF)

display(canvas)
thread.start()
"""
pass